# IBM AI Workflow Specialization Capstone #
## Assimilating The Business Opportunity And Articulate Testable Hypotheses ##

The management at AAVAIL wants to know whether their alternative approaches to generating revenue have ultimately been successful or not. They have a sizable amount of data over a reasonable amount of time, but aren't quite sure what to make of said data. They want a predictive model that will be able to predict the revenue for the following month, and additionally they want the ability for the service to predict projected revenue for specific countries in the dataset. 

The results are expected to be more accurate than the current model that they are using. 

### Potential hypotheses ###

There are a number of potential hypotheses that could be tested. It ultimately depends on the quality and complexity of the data that was collected. 
<ul>
<li>The amount of revenue generated for each month can be modelled as a Bayesian model, where the revenue for each month is dependent on the revenue of the previous month(s) and several other factors.</li>
<li>Revenue per month can also be correlated with the amount of content streamed per each individual user and average time of engagement with content on the platform in the previous month(s).</li>
<li>Revenue per month can also be correlated with the average amount of unique content streamed per each individual user in each of the previous month(s).</li>
</ul>

### Data needed to address each hypothesis ###

Ultimately the data that we need to address each of the hypotheses are as follows:

<ul>
<li>For modelling the amount of revenue generated for each month, we need the time of each stream aggregated by month and the total amount of money spent per invoice per month.</li>
<li>Modelling the amount of content streamed per each individual user simply requires counting all of the entries in a database and grouping them by user_id.</li>
<li>Calculating the average time of engagement with content on the platform is highly dependent on the amount of data that we have available. A simple start and end times for each stream might suffice, but in the case where that data isn't available, a good fallback would be the length/runtime of each piece of content that the user purchased.</li>
<li>Calculating the average amount of unique content streamed per each individual user simply requires aggregating over the dataset and grouping by the content_id and the user_id.</li>
</ul>

## Data ingestion ##

In [1]:
%%writefile load_data.py
import json
import glob
import pandas as pd
def load_data(pathname):
#load data
    data = []
    for filename in glob.glob(f"{pathname}/*.json"):
        with open(filename, "r") as file:
            for line in file:
                entry = json.loads(line)
                data += entry
    df = pd.DataFrame(data)
    return df

Overwriting load_data.py


In [4]:
#do a bit of EDA on the data before we can 
import pandas as pd
from load_data import load_data
DATA_DIR = "cs-train"
df = load_data(DATA_DIR)

There are several missing stream_ids even though there are two columns both with the name StreamID. While this isn't the end of the world, it does give pause as to see what other things are missing in the dataset. 

We are also missing quite a number of customer_ids as well, which isn't a bad thing since we can simply reconstruct the number of users based on each individual invoice. It is a big deal, however, in terms of the hypotheses I had proposed above.

In [55]:
#get the number of columns where both stream id columns are empty
print(df.columns)
print(len(df[(df["stream_id"].isna())&(df["StreamID"].isna())]))
#we have the full price data for each purchase in the dataset
print(len(df[(~df["total_price"].isna())|(~df["price"].isna())]))
#no for the most concerning bit: how many customer ids are there that are missing?
print(len(df[df["customer_id"].isna()]))

#however, there's another interesting question to ask: how many unique invoice_ids are there that have customer ids?
print(df[(df["customer_id"].isna())]["invoice"].unique())

print(len(df[(~df["times_viewed"].isna())|(~df["TimesViewed"].isna())]))
print(df[~df["times_viewed"].isna()]["times_viewed"])
print(df[~df["TimesViewed"].isna()]["TimesViewed"])
df["times_viewed"] = df.apply(lambda x: x["TimesViewed"] if x["TimesViewed"] != "NaN" else x["times_viewed"], axis=1)
print(df[df["times_viewed"].isna()])

Index(['country', 'customer_id', 'invoice', 'price', 'stream_id',
       'times_viewed', 'year', 'month', 'day', 'total_price', 'StreamID',
       'TimesViewed'],
      dtype='object')
292297
815011
189762
['489597' '489596' '489642' ... '562134' '562135' '562164']
522714
Series([], Name: times_viewed, dtype: float64)
45228      1.0
45229      1.0
45230     12.0
45231      1.0
45232      1.0
          ... 
567937     2.0
567938     2.0
567939     1.0
567940     5.0
567941     1.0
Name: TimesViewed, Length: 522714, dtype: float64
               country  customer_id invoice  price stream_id  times_viewed  \
0       United Kingdom      13085.0  489434   6.95       NaN           NaN   
1       United Kingdom          NaN  489597   8.65       NaN           NaN   
2       United Kingdom          NaN  489597   1.70       NaN           NaN   
3       United Kingdom          NaN  489597   1.70       NaN           NaN   
4       United Kingdom          NaN  489597   0.87       NaN           NaN 